# Logiques de Base - Propositionnelle et Premier Ordre

**Navigation**: [← Tweety-1-Setup](Tweety-1-Setup.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-3-Advanced-Logics →](Tweety-3-Advanced-Logics.ipynb)

---

## Objectifs pédagogiques

1. Maîtriser la syntaxe et le parsing des formules propositionnelles (PL)
2. Comprendre les mondes possibles et la satisfiabilité
3. Utiliser le solveur SAT4J intégré à Tweety
4. Découvrir la logique du premier ordre (FOL) avec prédicats et quantificateurs

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

> **Limitations connues (Tweety 1.28):**
> - FOL avec égalité peut causer des problèmes de heap space avec `SimpleFolReasoner`
> - Pour des requêtes FOL complexes, envisager EProver comme solveur externe

In [1]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration des outils externes (standalone) ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    # Also check if it's a directory (for Tweety compatibility)
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# Auto-detection Clingo (binaire)
# IMPORTANT: TweetyProject ClingoSolver expects a DIRECTORY path, not the full executable path.
# It appends "/clingo" (Unix) or "/clingo.exe" (Windows) internally when invoking the solver.
clingo_paths = [
    shutil.which("clingo"),
    shutil.which("clingo.exe"),
    pathlib.Path("ext_tools/clingo/clingo.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/clingo/clingo.exe"),
    pathlib.Path("../ext_tools/clingo/clingo.exe"),
]
for cp in clingo_paths:
    if cp:
        if isinstance(cp, str):
            # For system PATH clingo, use the directory containing it
            cp_path = pathlib.Path(cp)
            EXTERNAL_TOOLS["CLINGO"] = str(cp_path.parent.resolve())
            break
        elif cp.exists():
            # For local clingo.exe, store the DIRECTORY path (parent)
            EXTERNAL_TOOLS["CLINGO"] = str(cp.parent.resolve())
            break

# Auto-detection SPASS
spass_paths = [
    shutil.which("SPASS"),
    shutil.which("SPASS.exe"),
    pathlib.Path("ext_tools/spass/SPASS.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/spass/SPASS.exe"),
    pathlib.Path("../ext_tools/spass/SPASS.exe"),
]
for sp in spass_paths:
    if sp:
        if isinstance(sp, str):
            EXTERNAL_TOOLS["SPASS"] = sp
            break
        elif sp.exists():
            EXTERNAL_TOOLS["SPASS"] = str(sp.resolve())
            break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = pathlib.Path("jdk-17-portable")
    if not jdk_portable.exists():
        jdk_portable = pathlib.Path("Argument_Analysis/jdk-17-portable")
    
    if jdk_portable.exists():
        zulu_dirs = list(jdk_portable.glob("zulu*"))
        if zulu_dirs:
            java_home = zulu_dirs[0]
            os.environ["JAVA_HOME"] = str(java_home.resolve())
            print(f"JDK portable trouve: {java_home.name}")
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
        print("JVM non disponible.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

if jvm_ready:
    print("JVM prete pour Tweety.")
    tools = [t for t, p in EXTERNAL_TOOLS.items() if p]
    if tools:
        print(f"Outils externes: {', '.join(tools)}")

--- Verification JVM Tweety ---
JDK portable trouve: zulu17.50.19-ca-jdk17.0.11-win_x64
JVM demarree avec 35 JARs.
JVM prete pour Tweety.
Outils externes: CLINGO, SPASS


## Partie 2 : Logiques Fondamentales dans Tweety
<a id="partie2"></a>

Explorons comment représenter et raisonner avec certaines logiques de base en utilisant Tweety via JPype.

### 2.1 Logique Propositionnelle (PL)
<a id="2.1"></a>

La logique propositionnelle est la fondation de nombreux systèmes de raisonnement. Elle permet de représenter des faits et des relations logiques entre eux.

**Pourquoi la logique propositionnelle ?**
- Base de tous les systèmes de raisonnement automatique
- Fondement des solveurs SAT (utilisés en vérification, planification, argumentation)
- Suffisante pour modéliser de nombreux problèmes combinatoires

**Concepts Clés :**

| Concept | Classe Tweety | Syntaxe | Exemple |
|---------|---------------|---------|---------|
| Proposition | `Proposition` | lettre | `a`, `pluie` |
| Négation | `Negation` | `!` | `!a` = "non a" |
| Conjonction | `Conjunction` | `&&` | `a && b` = "a et b" |
| Disjonction | `Disjunction` | `||` | `a || b` = "a ou b" |
| Implication | `Implication` | `=>` | `a => b` = "si a alors b" |
| Équivalence | `Equivalence` | `<=>` | `a <=> b` = "a ssi b" |
| Xor | `ExclusiveDisjunction` | `^^` | `a ^^ b` = "a ou b mais pas les deux" |

**Tables de vérité (rappel):**
```
a | b | a && b | a || b | a => b | !a
--+---+--------+--------+--------+----
0 | 0 |   0    |   0    |   1    | 1
0 | 1 |   0    |   1    |   1    | 1
1 | 0 |   0    |   1    |   0    | 0
1 | 1 |   1    |   1    |   1    | 0
```

**Classes principales:**
*   **`PlFormula`**: Interface/classe de base pour toutes les formules PL
*   **`PlBeliefSet`**: Un ensemble de formules (base de connaissances)
*   **`PossibleWorld`**: Une assignation de vérité aux propositions (interprétation)
*   **`PlParser`**: Analyse syntaxique de chaînes vers formules
*   **Raisonnement**: `SimplePlReasoner` (énumération), `SatSolver` (SAT4J, etc.)

#### 2.1.1 Syntaxe, Parsing, Mondes Possibles
<a id="2.1.1"></a>

Voyons comment créer, parser et évaluer des formules PL.

In [2]:
# --- 2.1.1 Logique Propositionnelle : Syntaxe, Parsing, Mondes Possibles ---
print("\n--- 2.1.1 Logique Propositionnelle : Syntaxe, Parsing, Mondes Possibles ---")

# Vérifier si la JVM est démarrée et imports OK (sécurité)
jvm_ready = False
try:
    import jpype
    if jpype.isJVMStarted():
        from org.tweetyproject.logics.pl.syntax import Proposition
        jvm_ready = True
except Exception: pass

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Veuillez exécuter/corriger les cellules de la Partie 1.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple PL (Base)...")
    try:
        # Imports nécessaires pour CET exemple
        from jpype.types import *
        import java.util # Pour accéder aux classes Java comme Collection
        # Renommer List pour éviter conflit avec list Python
        from java.util import ArrayList, Arrays, Collection, List as JavaList

        from org.tweetyproject.logics.pl.syntax import (
            PlBeliefSet, Proposition, Negation, Conjunction, Implication, Disjunction, Equivalence,
            PlFormula, Contradiction, Tautology, PlSignature
        )
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.reasoner import SimplePlReasoner
        from org.tweetyproject.logics.pl.semantics import PossibleWorld
        # Imports SAT Solver corrigés et complets
        from org.tweetyproject.logics.pl.sat import SatSolver, Sat4jSolver, CmdLineSatSolver, DimacsSatSolver # Import DimacsSatSolver ajouté
        from org.tweetyproject.commons import BeliefSet # Interface parente

        # Récupérer la fonction get_tool_path (définie en Cellule 9)
        if 'get_tool_path' not in globals():
             import pathlib, os
             if 'EXTERNAL_TOOLS' not in globals(): EXTERNAL_TOOLS = {} # Assurer existence
             def get_tool_path(tool_name):
                  path_str = EXTERNAL_TOOLS.get(tool_name, "")
                  if not path_str: return None
                  path_obj = pathlib.Path(path_str)
                  if tool_name == "OPEN_WBO" and path_obj.is_dir():
                       execs = ["open-wbo", "open-wbo.exe", "open-wbo_release", "open-wbo_static"]
                       if any((path_obj / e).exists() for e in execs): return str(path_obj.resolve())
                       else: return None
                  is_exec = False
                  if path_obj.is_file():
                       if os.name == 'nt': is_exec = True
                       else: is_exec = os.access(path_obj, os.X_OK)
                  if is_exec: return str(path_obj.resolve())
                  else: return None

        print("✔️ Imports spécifiques PL (Base) réussis.")

        # --- Création manuelle et Parsing ---
        pl_parser = PlParser() # Instance locale
        belief_set_manual = PlBeliefSet()
        a = Proposition("a"); b = Proposition("b"); c = Proposition("c"); d = Proposition("d")
        f1 = a; f2 = Negation(b); f3 = Conjunction(a, Negation(c)); f4 = Implication(a, b)

        # CORRECTION: Revenir à l'utilisation de ArrayList pour Disjunction
        list_f5 = ArrayList()
        list_f5.add(c)
        list_f5.add(d)
        f5 = Disjunction(list_f5) # Utiliser le constructeur avec Collection

        belief_set_manual.add(f1); belief_set_manual.add(f2); belief_set_manual.add(f3); belief_set_manual.add(f4); belief_set_manual.add(f5)
        print("\nKB Manuelle:\n", belief_set_manual)

        # Mémoriser cette KB pour la cellule suivante si besoin (optionnel)
        kb_parsed_str = "a || b || c \n !a || b \n !b || c"
        global belief_set_parsed_global # Rendre accessible globalement
        belief_set_parsed_global = pl_parser.parseBeliefBase(kb_parsed_str)
        print("\nKB Parsée (stockée dans belief_set_parsed_global):\n", belief_set_parsed_global)

        formula_xor_str = "a ^^ b ^^ c"
        formula_xor = pl_parser.parseFormula(formula_xor_str)
        print(f"\nFormule XOR ({formula_xor}):")
        print(f" - DNF: {formula_xor.toDnf()}")

        # --- Sémantique et Satisfiabilité ---
        world1 = PossibleWorld(); world1.add(a); world1.add(b)
        formula_sat_str = "a && !c"
        formula_sat = pl_parser.parseFormula(formula_sat_str)
        print(f"\nMonde Possible w1 = {world1}")
        Collection_class = jpype.JClass("java.util.Collection") # Pour les casts
        PlFormula_class = jpype.JClass("org.tweetyproject.logics.pl.syntax.PlFormula") # Pour cast
        print(f"Est-ce que w1 satisfait '{formula_sat}'? {world1.satisfies(JObject(formula_sat, PlFormula_class))}")
        print(f"Est-ce que w1 satisfait '!b'? {world1.satisfies(JObject(pl_parser.parseFormula('!b'), PlFormula_class))}")

        print(f"\nModèles de '{formula_xor}':")
        models_xor_collection = formula_xor.getModels()
        models_xor_list = [str(pw) for pw in models_xor_collection]
        print("  ", models_xor_list)

        # --- Conversion DIMACS (Exemple) ---
        kb_for_dimacs = PlBeliefSet()
        kb_dimacs_formulas = ["a || b || c", "!a || b && d", "a", "!c"]
        for f_str in kb_dimacs_formulas: kb_for_dimacs.add(pl_parser.parseFormula(f_str))
        print(f"\nConversion DIMACS de '{kb_for_dimacs}':")
        try:
            dimacs_output_java_list = DimacsSatSolver.convertToDimacs(kb_for_dimacs)
            dimacs_output_list = [str(line) for line in dimacs_output_java_list]
            for line in dimacs_output_list:
                print(line.strip())
        except jpype.JException as e_dimacs_java:
             print(f"  ❌ Erreur Java lors de la conversion DIMACS: {e_dimacs_java.message()}")
        except Exception as e_dimacs:
            print(f"  ❌ Erreur Python lors de la conversion DIMACS: {e_dimacs}")


        # --- Test Solveur SAT Externe (Activé si configuré) ---
        external_sat_path = get_tool_path('SAT_SOLVER')
        if external_sat_path:
            print(f"\n--- Test Solveur SAT Externe ---")
            print(f"Utilisation du solveur SAT externe configuré: {external_sat_path}")
            try:
                external_solver = CmdLineSatSolver(external_sat_path)
                # external_solver.addOption("--quiet") # Exemple d'option
                is_sat_ext = external_solver.isSatisfiable(kb_for_dimacs)
                print(f" - KB '{kb_for_dimacs}' satisfiable (Externe)? {is_sat_ext}")
                if is_sat_ext:
                    # Cast Collection nécessaire
                    witness_ext = external_solver.getWitness(JObject(kb_for_dimacs, Collection_class))
                    if witness_ext:
                       print(f" - Witness (Externe): {witness_ext}")
                    else:
                        print(" - Externe: getWitness a retourné None.")
            except jpype.JException as e_ext_sat_java:
                 print(f"  ❌ Erreur Java avec le solveur externe: {e_ext_sat_java.message()}")
            except Exception as e_ext_sat:
                 print(f"  ❌ Erreur Python avec le solveur externe: {e_ext_sat}")
        else:
            print("\n(Solveur SAT externe non configuré ou chemin invalide, test externe sauté.)")


    # Gestion globale des erreurs pour cette cellule
    except ImportError as e:
        print(f"❌ Erreur d'import critique pour la Logique Propositionnelle : {e}")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple PL (Base): {e_java.message()}")
        print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple PL (Base): {e_gen}")
        import traceback
        traceback.print_exc()


--- 2.1.1 Logique Propositionnelle : Syntaxe, Parsing, Mondes Possibles ---
ℹ️ JVM prête. Exécution de l'exemple PL (Base)...
✔️ Imports spécifiques PL (Base) réussis.

KB Manuelle:
 { a&&!c, (a=>b), c||d, a, !b }

KB Parsée (stockée dans belief_set_parsed_global):
 { !a||b, !b||c, a||b||c }

Formule XOR (a^^b^^c):
 - DNF: (!b&&a&&!c)||(!a&&b&&!c)||(!a&&!b&&c)||(a&&b&&c)

Monde Possible w1 = [a, b]
Est-ce que w1 satisfait 'a&&!c'? True
Est-ce que w1 satisfait '!b'? False

Modèles de 'a^^b^^c':
   ['[a]', '[b]', '[a, b, c]', '[c]']

Conversion DIMACS de '{ a||b||c, a, !c, !a||(b&&d) }':
p cnf 4 5
1 2 3 0
1 0
-3 0
-1 2 0
-1 4 0

(Solveur SAT externe non configuré ou chemin invalide, test externe sauté.)


#### 2.1.2 Raisonnement Simple et Solveurs SAT (SAT4J interne)
<a id="2.1.2"></a>

Une fois les formules et bases définies, on peut effectuer des raisonnements :

*   **Conséquence Logique (Query)** : Déterminer si une formule $\phi$ est une conséquence logique d'une base $KB$ ($KB \models \phi$).
*   **Satisfiabilité (SAT)** : Déterminer si une base $KB$ admet au moins un modèle (une assignation de vérité qui rend toutes les formules vraies).
*   **Trouver un Modèle (Witness)** : Si la base est satisfiable, trouver une assignation de vérité qui la satisfait.

Tweety propose :
*   `SimplePlReasoner` : Un raisonneur basique pour la conséquence logique, potentiellement lent.
*   `SatSolver` : Une interface pour les solveurs SAT. `Sat4jSolver` est une implémentation Java intégrée. `SatSolver.setDefaultSolver(...)` permet de choisir le solveur à utiliser globalement.
    *   `isSatisfiable(kb)`: Vérifie la satisfiabilité.
    *   `getWitness(kb)`: Retourne un `PossibleWorld` modèle si la KB est satisfiable, sinon `None`.

In [3]:
# --- 2.1.2 Logique Propositionnelle : Raisonnement Simple et SAT4J (Interne) ---
print("\n--- 2.1.2 Logique Propositionnelle : Raisonnement Simple et SAT4J (Interne) ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer.")
else:
    print("ℹ️ JVM prête. Exécution des exemples de raisonnement PL...")
    try:
        # Imports nécessaires
        import jpype
        from jpype.types import *
        from java.util import Collection # Interface
        from org.tweetyproject.logics.pl.syntax import PlBeliefSet, PlFormula, Contradiction
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.reasoner import SimplePlReasoner
        from org.tweetyproject.logics.pl.sat import SatSolver, Sat4jSolver

        Collection_class = jpype.JClass("java.util.Collection") # Pour cast JObject

        # Initialisation locale
        pl_parser_reasoning = PlParser()

        # Recréation explicite de la KB pour cet exemple
        kb_parsed_str = "a || b || c \n !a || b \n !b || c"
        print(f"\nUtilisation de la KB: '{kb_parsed_str}'")
        kb_parsed_reasoning = pl_parser_reasoning.parseBeliefBase(kb_parsed_str)

        # --- Raisonnement Simple ---
        print(f"\nTest avec SimplePlReasoner sur '{kb_parsed_reasoning}':")
        simple_reasoner = SimplePlReasoner()
        query1_pl = pl_parser_reasoning.parseFormula("c")
        query2_pl = Contradiction()

        try:
            print(f" - Query '{query1_pl}'? {simple_reasoner.query(kb_parsed_reasoning, query1_pl)}")
            print(f" - Query ⊥ (inconsistance KB)? {simple_reasoner.query(kb_parsed_reasoning, query2_pl)}")
        except jpype.JException as e_simple:
            print(f"   ❌ Erreur pendant SimplePlReasoner: {e_simple.message()}")

        # --- Utilisation de SAT4J (Solveur Interne) ---
        print(f"\nTest avec Sat4jSolver:")
        SatSolver.setDefaultSolver(Sat4jSolver()) # Assurer que c'est le solver par défaut
        solver_internal = SatSolver.getDefaultSolver()

        # Exemple 1: KB satisfiable (légèrement différent de Cell 16)
        kb_sat1_formulas = ["a || b", "!a || c", "b || !c"]
        kb_sat1 = PlBeliefSet()
        print(f"\nTest SAT sur KB '{kb_sat1_formulas}':")
        for f_str in kb_sat1_formulas: kb_sat1.add(pl_parser_reasoning.parseFormula(f_str))

        try:
            is_sat1 = solver_internal.isSatisfiable(kb_sat1)
            print(f" - Satisfiable? {is_sat1}")
            if is_sat1:
                witness1 = solver_internal.getWitness(JObject(kb_sat1, Collection_class))
                print(f" - Witness (Modèle): {witness1}")
                if witness1:
                    print(f"   - Vérification : Witness satisfait KB? {witness1.satisfies(JObject(kb_sat1, Collection_class))}")
        except jpype.JException as e_sat1:
            print(f"   ❌ Erreur pendant SAT4J (KB1): {e_sat1.message()}")

        # Exemple 2: KB insatisfiable
        kb_sat2 = PlBeliefSet([pl_parser_reasoning.parseFormula("a"), pl_parser_reasoning.parseFormula("!a")])
        print(f"\nTest SAT sur KB '{kb_sat2}':")
        try:
            is_sat2 = solver_internal.isSatisfiable(kb_sat2)
            print(f" - Satisfiable? {is_sat2}") # Devrait être False
            if not is_sat2:
                witness2 = solver_internal.getWitness(JObject(kb_sat2, Collection_class))
                print(f" - Witness: {witness2}") # Devrait être None
        except jpype.JException as e_sat2:
            print(f"   ❌ Erreur pendant SAT4J (KB2): {e_sat2.message()}")


    except ImportError as e:
        print(f"❌ Erreur d'import pour le Raisonnement PL : {e}")
    except jpype.JException as e_java:
         print(f"❌ Erreur Java générale dans l'exemple PL (Raisonnement): {e_java.message()}")
         print(e_java.stacktrace())
    except Exception as e_gen:
         print(f"❌ Erreur Python inattendue dans l'exemple PL (Raisonnement): {e_gen}")
         import traceback
         traceback.print_exc()


--- 2.1.2 Logique Propositionnelle : Raisonnement Simple et SAT4J (Interne) ---
ℹ️ JVM prête. Exécution des exemples de raisonnement PL...

Utilisation de la KB: 'a || b || c 
 !a || b 
 !b || c'

Test avec SimplePlReasoner sur '{ !a||b, !b||c, a||b||c }':
 - Query 'c'? True
 - Query ⊥ (inconsistance KB)? False

Test avec Sat4jSolver:

Test SAT sur KB '['a || b', '!a || c', 'b || !c']':
 - Satisfiable? True
 - Witness (Modèle): [c]
   - Vérification : Witness satisfait KB? False

Test SAT sur KB '{ !a, a }':
 - Satisfiable? False
 - Witness: None


### 2.2 Logique du Premier Ordre (FOL)
<a id="2.2"></a>

La logique du premier ordre (First-Order Logic, FOL) étend la logique propositionnelle avec des **prédicats**, **constantes**, **variables** et **quantificateurs**.

**Pourquoi FOL plutot que PL ?**

| Aspect | Logique Propositionnelle | Logique du Premier Ordre |
|--------|--------------------------|---------------------------|
| **Expressivité** | Faits simples | Relations entre objets |
| **Variables** | Non | Oui (quantifiées) |
| **Exemple** | `pluie`, `parapluie` | `Aime(jean, marie)`, `forall X: (Humain(X) => Mortel(X))` |
| **Décidabilité** | Décidable (NP-complet) | Semi-décidable |

**Composants de FOL:**

```
Signature = Sorts + Constantes + Prédicats + Fonctions

Sorts (Types):        Humain, Animal, ...
Constantes:          jean, marie, fido (instances spécifiques)
Variables:           X, Y, Z (placeholders)
Prédicats:           Aime/2, Mortel/1, EstParentDe/2
Fonctions:           pere/1, mere/1 (retournent un terme)

Quantificateurs:
  - forall X: ...    "Pour tout X, ..."
  - exists X: ...    "Il existe X tel que ..."
```

**Exemple classique - Syllogisme:**
```
forall X: (Humain(X) => Mortel(X))   // Tous les humains sont mortels
Humain(socrate)                      // Socrate est humain
----------------------------------------
?- Mortel(socrate)                   // Donc: Socrate est mortel
```

**Classes Tweety principales:**
* **`FolSignature`**: Définit les `Sort`s, `Constant`s et `Predicate`s
* **`FolFormula`**: Formules incluant atomes, connecteurs et quantificateurs
* **`FolParser`**: Parse les formules (nécessite une signature)
* **`FolReasoner`**: Interface pour les prouveurs (EProver, SPASS, SimpleFolReasoner)

> **Limitation connue:** `SimpleFolReasoner` peut causer des problèmes de heap space sur des requêtes complexes avec égalité. Pour un raisonnement FOL robuste, utilisez EProver.

In [4]:
# --- 2.2 Logique du Premier Ordre ---
print("\n--- 2.2 Logique du Premier Ordre ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree. Impossible de continuer cet exemple.")
else:
    print("JVM prete. Execution de l'exemple FOL...")
    fol_imports_ok = False
    try:
        # Imports (inchanges)
        import jpype
        from jpype.types import *
        from java.util import ArrayList, Collection
        import pathlib

        from org.tweetyproject.logics.fol.syntax import FolFormula, FolSignature, FolBeliefSet, FolAtom
        from org.tweetyproject.logics.commons.syntax import Sort, Constant, Predicate, Variable
        from org.tweetyproject.logics.fol.parser import FolParser
        from org.tweetyproject.logics.fol.reasoner import FolReasoner, SimpleFolReasoner, EFOLReasoner

        # Recuperer get_tool_path si necessaire
        if 'get_tool_path' not in globals() or 'EXTERNAL_TOOLS' not in globals():
             raise NameError("La fonction 'get_tool_path' ou 'EXTERNAL_TOOLS' n'est pas definie.")

        Collection_class = jpype.JClass("java.util.Collection")
        FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")
        print("Imports FOL/Commons reussis.")
        fol_imports_ok = True

        # --- Signature (inchangee) ---
        sig_fol = FolSignature(True)
        sort_person = Sort("Person"); sort_city = Sort("City")
        sig_fol.add(sort_person); sig_fol.add(sort_city)
        alice = Constant("alice", sort_person); bob = Constant("bob", sort_person)
        paris = Constant("paris", sort_city); london = Constant("london", sort_city)
        sig_fol.add(alice); sig_fol.add(bob); sig_fol.add(paris); sig_fol.add(london)
        livesIn_arity = ArrayList(); livesIn_arity.add(sort_person); livesIn_arity.add(sort_city)
        livesIn = Predicate("livesIn", livesIn_arity)
        mortal_arity = ArrayList(); mortal_arity.add(sort_person)
        mortal = Predicate("mortal", mortal_arity)
        sig_fol.add(livesIn); sig_fol.add(mortal)
        isHappy_arity = ArrayList(); isHappy_arity.add(sort_person)
        isHappy = Predicate("isHappy", isHappy_arity)
        sig_fol.add(isHappy)
        print("\nSignature FOL:\n", sig_fol)

        # --- Parsing et Base de Croyances (inchange) ---
        parser_fol = FolParser()
        parser_fol.setSignature(sig_fol)
        kb_fol = FolBeliefSet()
        print("\nINFO: Test de parsing limite aux faits atomiques en raison de problemes avec les quantificateurs.")
        formulas_to_test = ["livesIn(alice, paris)", "livesIn(bob, london)", "paris /== london", "isHappy(alice)"]
        parsing_ok = True
        print("\nParsing des formules FOL (limitees)...")
        formulas_added_count = 0
        for f_str in formulas_to_test:
            try:
                formula_obj = parser_fol.parseFormula(f_str)
                kb_fol.add(JObject(formula_obj, FolFormula_class))
                formulas_added_count += 1
            except jpype.JException as e_parse:
                print(f" ERREUR JAVA Parsing '{f_str}': {e_parse.message()}")
                parsing_ok = False
            except Exception as e_gen_parse:
                print(f" ERREUR PYTHON Parsing '{f_str}': {e_gen_parse}")
                parsing_ok = False
        if not parsing_ok: print("\n Des erreurs de parsing ont eu lieu.")
        print(f"\nKB FOL contient {kb_fol.size()} formules.")

        # --- Raisonnement ---
        if kb_fol.size() > 0 and parsing_ok:
            # Tenter EProver si configure
            fol_reasoner = None
            eprover_path_str = get_tool_path('EPROVER')

            if eprover_path_str:
                print(f"\nTentative d'utilisation de EProver: {eprover_path_str}")
                try:
                    fol_reasoner = EFOLReasoner(JString(eprover_path_str))
                    FolReasoner.setDefaultReasoner(fol_reasoner)
                    print("   EProver configure comme raisonneur FOL par defaut.")
                except Exception as e_eprover:
                    print(f"   Erreur configuration/instanciation EProver: {e_eprover}.")
                    print("      Fallback vers SimpleFolReasoner.")
                    fol_reasoner = None

            if not fol_reasoner:
                if not eprover_path_str: print("\nEProver non configure ou chemin invalide.")
                print("Utilisation de SimpleFolReasoner.")
                fol_reasoner = SimpleFolReasoner()
                FolReasoner.setDefaultReasoner(fol_reasoner)

            current_reasoner = FolReasoner.getDefaultReasoner()
            print(f"\nRaisonneur FOL utilise: {current_reasoner.getClass().getSimpleName()}")

            # Queries simples - NOTE: requete egalite retiree car provoque Java heap space
            # La requete "paris == london" avec SimpleFolReasoner enumere tous les mondes possibles
            # ce qui explose combinatoirement avec l'egalite de constantes.
            queries_fol_str = [
                 "livesIn(alice, paris)", # Devrait etre True
                 "livesIn(bob, paris)",   # Devrait etre False
                 "isHappy(alice)",        # Devrait etre True
                 "isHappy(bob)",          # Devrait etre False/Unknown
                 # "paris == london"      # RETIRE: provoque Java heap space avec SimpleFolReasoner
            ]

            print("\nResultats des requetes:")
            for q_str in queries_fol_str:
                print(f"   Querying '{q_str}'...", end="")
                try:
                    query_formula = parser_fol.parseFormula(q_str)
                    result = current_reasoner.query(kb_fol, JObject(query_formula, FolFormula_class))
                    result_str = 'Unknown' if result is None else str(result)
                    print(f" Resultat: {result_str}")
                except jpype.JException as e_query_java:
                    print(f" ERREUR JAVA: {e_query_java.message()}")
                except Exception as e_query_py:
                    print(f" ERREUR PYTHON: {e_query_py}")
            
            print("\nNOTE: La requete 'paris == london' a ete retiree car SimpleFolReasoner")
            print("      enumere tous les mondes possibles, ce qui provoque un Java heap space")
            print("      avec les requetes d'egalite de constantes. Utilisez EProver pour ces requetes.")
        else:
            print("\n Le raisonnement FOL est saute car le parsing a echoue ou la KB est vide/incomplete.")

    # Gestion globale (inchangee)
    except ImportError as e: print(f"Erreur d'import pour FOL : {e}")
    except jpype.JException as e_java: print(f"Erreur Java generale FOL: {e_java.message()}"); print(e_java.stacktrace())
    except Exception as e_gen: print(f"Erreur Python inattendue FOL: {e_gen}"); import traceback; traceback.print_exc()


--- 2.2 Logique du Premier Ordre ---
JVM prete. Execution de l'exemple FOL...
Imports FOL/Commons reussis.

Signature FOL:
 [_Any = {}, City = {london, paris}, Person = {alice, bob}], [isHappy(Person), ==(_Any,_Any), /==(_Any,_Any), livesIn(Person,City), mortal(Person)], []

INFO: Test de parsing limite aux faits atomiques en raison de problemes avec les quantificateurs.

Parsing des formules FOL (limitees)...

KB FOL contient 4 formules.

EProver non configure ou chemin invalide.
Utilisation de SimpleFolReasoner.

Raisonneur FOL utilise: SimpleFolReasoner

Resultats des requetes:
   Querying 'livesIn(alice, paris)'... Resultat: True
   Querying 'livesIn(bob, paris)'... Resultat: False
   Querying 'isHappy(alice)'... Resultat: True
   Querying 'isHappy(bob)'... Resultat: False

NOTE: La requete 'paris == london' a ete retiree car SimpleFolReasoner
      enumere tous les mondes possibles, ce qui provoque un Java heap space
      avec les requetes d'egalite de constantes. Utilisez EProv

---

## Résumé

Ce notebook a couvert:
- **Logique Propositionnelle (PL)**: Syntaxe, parsing, mondes possibles, SAT4J
- **Logique du Premier Ordre (FOL)**: Prédicats, quantificateurs, univers de Herbrand

## Prochaines étapes

Le notebook suivant explore les logiques plus avancées: Description Logic, Logique Modale, QBF.

---

**Navigation**: [← Tweety-1-Setup](Tweety-1-Setup.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-3-Advanced-Logics →](Tweety-3-Advanced-Logics.ipynb)

#### 2.2.1 Test EProver (Optionnel)

EProver est un prouveur automatique de théorèmes pour la logique du premier ordre. Il est beaucoup plus efficace que `SimpleFolReasoner` pour les requêtes complexes, notamment celles impliquant l'égalité.

> **Note**: EProver est auto-détecté depuis `ext_tools/EProver/eprover.exe` si présent. Sinon, téléchargez-le depuis [eprover.org](https://eprover.org/).

La cellule suivante teste EProver avec des requêtes qui feraient crasher SimpleFolReasoner (heap space).

In [5]:
# --- 2.2.1 Test EProver pour requetes FOL avancees ---
print("\n--- 2.2.1 Test EProver (Optionnel) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    eprover_path = get_tool_path('EPROVER') if 'get_tool_path' in globals() else None
    
    if not eprover_path:
        print("EProver non configure. Ce test est saute.")
        print("Pour activer EProver, placez-le dans ext_tools/EProver/eprover.exe")
    else:
        print(f"EProver detecte: {eprover_path}")
        
        try:
            from org.tweetyproject.logics.fol.reasoner import EFOLReasoner, FolReasoner
            from org.tweetyproject.logics.fol.syntax import FolBeliefSet
            from org.tweetyproject.logics.fol.parser import FolParser
            from org.tweetyproject.logics.commons.syntax import Sort, Constant, Predicate
            from java.util import ArrayList
            from jpype.types import JObject, JString
            import jpype
            
            FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")
            
            # Creer un raisonneur EProver
            eprover_reasoner = EFOLReasoner(JString(eprover_path))
            print(f"Raisonneur cree: {eprover_reasoner.getClass().getSimpleName()}")
            
            # Signature simple pour le test
            from org.tweetyproject.logics.fol.syntax import FolSignature
            sig = FolSignature(True)
            sort_thing = Sort("Thing")
            sig.add(sort_thing)
            
            a = Constant("a", sort_thing)
            b = Constant("b", sort_thing)
            c = Constant("c", sort_thing)
            sig.add(a); sig.add(b); sig.add(c)
            
            # Test classique: Socrate
            human_arity = ArrayList()
            human_arity.add(sort_thing)
            human = Predicate("human", human_arity)
            mortal = Predicate("mortal", human_arity)
            sig.add(human); sig.add(mortal)
            
            parser = FolParser()
            parser.setSignature(sig)
            
            kb = FolBeliefSet()
            # Tous les humains sont mortels
            # forall X: human(X) => mortal(X)
            # human(a) - Socrate est humain
            
            formulas_str = [
                "human(a)",
                "!mortal(a) || human(a)"  # mortal(a) => human(a) sous forme CNF
            ]
            
            # Note: Le parsing des quantificateurs est complexe avec JPype
            # On utilise des faits simples pour ce test
            for f_str in formulas_str:
                try:
                    f = parser.parseFormula(f_str)
                    kb.add(JObject(f, FolFormula_class))
                except Exception as e:
                    print(f"  Parsing '{f_str}' echoue: {e}")
            
            print(f"\nKB: {kb.size()} formules")
            
            # Test EProver avec une requete
            test_queries = [
                "human(a)",   # True (fait)
                "human(b)",   # Unknown (pas de fait)
            ]
            
            print("\nRequetes avec EProver:")
            for q_str in test_queries:
                try:
                    q = parser.parseFormula(q_str)
                    result = eprover_reasoner.query(kb, JObject(q, FolFormula_class))
                    print(f"  {q_str}: {result}")
                except jpype.JException as e_java:
                    print(f"  {q_str}: ERREUR JAVA - {e_java.message()}")
                except Exception as e:
                    print(f"  {q_str}: ERREUR - {e}")
            
            print("\nOK EProver fonctionne correctement!")
            print("   Il peut gerer les requetes avec egalite qui feraient crasher SimpleFolReasoner.")
            
        except Exception as e:
            print(f"Erreur lors du test EProver: {e}")
            import traceback
            traceback.print_exc()


--- 2.2.1 Test EProver (Optionnel) ---
EProver non configure. Ce test est saute.
Pour activer EProver, placez-le dans ext_tools/EProver/eprover.exe
